In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()

output_error = pd.DataFrame()

input_ = pd.read_excel('./1.category.xlsx',
                       header=0,
                       dtype=str).fillna('')

length = len(input_)

print('总数量：' + str(length))
print()

# = = = = = = = = = = = = = = =

from urllib.parse import quote
import requests
import json

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for i in range(length):
    work.put_nowait(i)

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        i = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for _ in range(101):
            try:
                jsn = '{"jsn":{"tab":"extras","groupname":"' + input_.loc[i, 'Category'] + '","nodetype":"groupname"}}'

                payload = {'func': 'navnode_fetch',
                           'payload': jsn,
                           'api_json_request': '1'}

                resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                     data=payload,
                                     headers=UA.get_User_Agent_Requests(),
                                     proxies=Proxy.get_Proxy_Requests()).text

                resp = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                html = resp['html_fill_sections']['navchildren[]']
                
                soup = BeautifulSoup(html, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                list_subcategory = [subcategory.strip() for subcategory in html.xpath('//td[@class="nlabel"]/a/text()')]

                # = = = = = = = = = = = = = = =
                
                list_url = ['https://www.rockauto.com'+url.strip() for url in html.xpath('//td[@class="niconspace ncollapsedicon"]/a/@href')]
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
                
            except:
                time.sleep(0.3)
                continue

        if crawler_status == 'ok':
            df_temp = pd.DataFrame({'Tab_Label': input_.loc[i, 'Tab_Label'],
                                    'No.': int(input_.loc[i, 'No.']),
                                    'Category': input_.loc[i, 'Category'],
                                    'Subcategory': list_subcategory,
                                    'Url': list_url})

            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
        else:
            df_temp = pd.DataFrame([{'Tab_Label': input_.loc[i, 'Tab_Label'],
                                     'No.': int(input_.loc[i, 'No.']),
                                     'Category': input_.loc[i, 'Category'],
                                     'Url': input_.loc[i, 'Url']}])
    
            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        print(input_.loc[i, 'Category'] +  '  <->  [' + crawler_status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['No.', 'Subcategory'], ascending=[True, True])
output_correct.to_excel('./2.subategory-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True])
    output_error.to_excel('./subategory_error.xlsx', index=False)
    print()
    print('爬虫存在error')
print()
print('爬虫结束')

总数量：26

Body & Lamp Assembly  <->  [ok] - 剩余数量：23 - 当前时间：17:00:50
Brake & Wheel Hub  <->  [ok] - 剩余数量：22 - 当前时间：17:00:51
Cooling System  <->  [ok] - 剩余数量：21 - 当前时间：17:00:53
Drivetrain  <->  [ok] - 剩余数量：20 - 当前时间：17:00:54
Electrical  <->  [ok] - 剩余数量：19 - 当前时间：17:00:55
Electrical-Bulb & Socket  <->  [ok] - 剩余数量：18 - 当前时间：17:00:58
Electrical-Connector  <->  [ok] - 剩余数量：17 - 当前时间：17:01:00
Electrical-Switch & Relay  <->  [ok] - 剩余数量：16 - 当前时间：17:01:02
Engine  <->  [ok] - 剩余数量：15 - 当前时间：17:01:04
Exhaust & Emission  <->  [ok] - 剩余数量：14 - 当前时间：17:01:07
Fuel & Air  <->  [ok] - 剩余数量：13 - 当前时间：17:01:09
Garage Equipment  <->  [ok] - 剩余数量：12 - 当前时间：17:01:11
Hardware  <->  [ok] - 剩余数量：11 - 当前时间：17:01:13
Heat & Air Conditioning  <->  [ok] - 剩余数量：10 - 当前时间：17:01:15
Hoses/Lines & Clamps  <->  [ok] - 剩余数量：9 - 当前时间：17:01:21
Apparel & Gifts  <->  [ok] - 剩余数量：8 - 当前时间：17:01:22
Ignition  <->  [ok] - 剩余数量：7 - 当前时间：17:01:23
Literature  <->  [ok] - 剩余数量：6 - 当前时间：17:01:24
Interior  <->  [ok] - 剩余数量：5 - 当前时间：17

KeyError: 'Year'